<a href="https://colab.research.google.com/github/sundarp17/semi-supervised/blob/main/train4_svc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:

train4 = pd.read_csv(r'/content/train_data4.csv')
test = pd.read_csv(r'/content/test_data.csv')

In [3]:

train4['Target'] = train4['Target'].replace(['Others'],'Invalid')


In [4]:
train4['Target'].unique()

array(['Invalid', 'Issue', 'Analysis', 'Facts', 'Conclusion',
       'Rule/Law/Holding'], dtype=object)

In [5]:
#cleaning
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()

def clean_text(text):
  text="".join([word.lower() for word in text if word not in string.punctuation])
  tokens = re.split('\W+',text)
  text = [wl.lemmatize(word) for word in tokens if word not in stopword]
  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer = clean_text)

train4_Xtfidf = tfidf_vect.fit_transform(train4['Sentence'])


In [7]:
test['Target']=test['Target'].replace(['Others'],'Invalid')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
test['Sentence'] = test['Sentence'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x for x in x.split() if x not in words))

In [8]:
unlabel_1 = test.loc[:100]
print("length of unlabel_1",len(unlabel_1))
unlabel_2 = test.loc[101:200]
print("length of unlabel_2",len(unlabel_2))
unlabel_3 = test.loc[201:300]
print("length of unlabel_3",len(unlabel_3))
unlabel_4 = test.loc[301:400]
print("length of unlabel_4",len(unlabel_4))
unlabel_5 = test.loc[401:]
print("length of unlabel_5",len(unlabel_5))

length of unlabel_1 101
length of unlabel_2 100
length of unlabel_3 100
length of unlabel_4 100
length of unlabel_5 120


In [9]:
unlabel_2.reset_index(inplace=True)
del unlabel_2['index']
unlabel_2.head()

,Unnamed: 0,Sentence,Target
0,1207,examination occurred night middle may testifie...,Invalid
1,3400,undisputed testimony reflects appellant drivin...,Facts
2,2072,appellant testify offer evidence behalf,Facts
3,517,56 uniform act regulating traffic highways gis...,Rule/Law/Holding
4,906,agreed accompany gave pawn ticket money reclai...,Facts


In [10]:
unlabel_3.reset_index(inplace=True)
del unlabel_3['index']
unlabel_3.head()

,Unnamed: 0,Sentence,Target
0,1882,find evidence amply sufficient sustain juryâs ...,Analysis
1,964,actions show thatâ,Invalid
2,1417,certification attending physician shown filed ...,Facts
3,2932,testified agent bland first left automobile wa...,Facts
4,1584,officer testified could smell odor alcohol app...,Facts


In [11]:
unlabel_4.reset_index(inplace=True)
del unlabel_4['index']
unlabel_4.head()

,Unnamed: 0,Sentence,Target
0,266,section 6 pl,Invalid
1,3469,amended answer appellant alleged bond invalid ...,Analysis
2,1471,evidence sufficient support conviction appella...,Analysis
3,1990,cases cited note 19,Invalid
4,2757,july 8 1960 order entered revoking said probat...,Facts


In [12]:
unlabel_5.reset_index(inplace=True)
del unlabel_5['index']
unlabel_5.head()

,Unnamed: 0,Sentence,Target
0,2979,overrule appellantâs contention court erred pe...,Analysis
1,2135,state regarded authority case,Invalid
2,1591,testimony adduced appellant mother lillie mae ...,Facts
3,658,appellant needle marks inside left elbow,Facts
4,2562,analysis contents 3 capsules chemist kenneth a...,Facts


In [13]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.calibration import CalibratedClassifierCV


X_train, x_val, Y_train, y_val = train_test_split(train4_Xtfidf,train4['Target'],test_size=0.26,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model=calibrator.fit(X_train, Y_train)
pred_svm = model.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6902654867256637
                  precision    recall  f1-score   support

        Analysis       0.53      0.40      0.46       127
      Conclusion       0.77      0.57      0.66        42
           Facts       0.74      0.92      0.82       487
         Invalid       0.54      0.46      0.50       128
           Issue       0.61      0.57      0.59        49
Rule/Law/Holding       0.76      0.23      0.35        71

        accuracy                           0.69       904
       macro avg       0.66      0.52      0.56       904
    weighted avg       0.68      0.69      0.67       904



In [14]:
x_un1 = tfidf_vect.transform(unlabel_1['Sentence'])
pred_unlabel_1 = model.predict_proba(x_un1)

In [15]:
import numpy as np
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_1:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 4, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2]
[0.9976446554667873, 0.9998857063356257, 0.9801852088244083, 0.9996778695426336, 0.9917547899098745, 0.9972964289546589, 0.9882755355552167, 0.9980808937941643, 0.9964583382232526, 0.9986752711001027, 0.9952845198647305, 0.988123185969496, 0.9984529568746784, 0.9902519824649393, 0.9989298894103654, 0.991984592021807, 0.999580119895415, 0.9999034223563306, 0.9863596500209243, 0.9955590372957238, 0.9958671898879797, 0.9848809402928099, 0.9995578025765453, 0.9975123016024655, 0.9967036702485855, 0.9845105069785309, 0.9908608755773808, 0.9887113159753773, 0.9990319088886848, 0.992719347802257, 0.9802894526716898, 0.9886459205824737, 0.9935631452938092, 0.993780705149323, 0.9997870814306608, 0.994237993102625, 0.9985296693132137, 0.9931592760433172, 0.9810637455663451]
[4, 5, 6, 11, 12, 18, 21, 23, 24, 29, 31, 34, 36, 37, 39, 40, 45, 48, 49, 50, 51, 52, 56, 60, 62, 69, 70, 7

In [16]:
unlabel_1=unlabel_1.loc[[4, 5, 6, 11, 12, 18, 21, 23, 24, 29, 31, 34, 36, 37, 39, 40, 45, 48, 49, 50, 51, 52, 56, 60, 62, 69, 70, 74, 76, 77, 79, 85, 87, 89, 94, 95, 96, 98, 100],:]
unlabel_1.head()

,Unnamed: 0,Sentence,Target
4,79,eligible leaves renew contacts family,Facts
5,1955,got downstairs ponce went car wife left home,Facts
6,324,defendant argues impossible reconcile plaintif...,Facts
11,1604,police notified deceased found lying mrs moers...,Facts
12,1102,shown vine street drug storeâs supply narcotic...,Facts


In [17]:
frame_1 = [train4,unlabel_1]
train4_1 = pd.concat(frame_1)
len(train4_1)

3515

In [18]:
x_train_1 = tfidf_vect.transform(train4_1['Sentence'])
x_train_1.shape
X_train, x_val, Y_train, y_val = train_test_split(x_train_1,train4_1['Target'],test_size=0.2,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_1=calibrator.fit(X_train, Y_train)
pred_svm = model_1.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6685633001422475
                  precision    recall  f1-score   support

        Analysis       0.48      0.44      0.46       103
      Conclusion       0.71      0.57      0.63        35
           Facts       0.75      0.90      0.82       351
         Invalid       0.53      0.48      0.50       113
           Issue       0.59      0.56      0.57        36
Rule/Law/Holding       0.58      0.22      0.31        65

        accuracy                           0.67       703
       macro avg       0.61      0.53      0.55       703
    weighted avg       0.65      0.67      0.65       703



In [19]:
x_un2 = tfidf_vect.transform(unlabel_2['Sentence'])
pred_unlabel_2 = model_1.predict_proba(x_un2)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_2:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 3, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2]
[0.9966675063769959, 0.9988668724158479, 0.9847597607868097, 0.9865962704306853, 0.9955920875631854, 0.9999545539567573, 0.9880845104127925, 0.9948831000760658, 0.9954402326984985, 0.9984732885237817, 0.9831299519881391, 0.9835884959725196, 0.9965166356576098, 0.999866656006759, 0.9992253833951891, 0.9998151089955913, 0.9952146811109668, 0.9995263992173956, 0.9965332083411933, 0.9994237519109345, 0.9822210032488562, 0.998826377283058, 0.999802915225826, 0.9986017702848874, 0.996923071075056, 0.9894281282963315, 0.9848427423979457, 0.9974546486540673, 0.9845717641087665, 0.9976371830327294, 0.9966563389238702, 0.9955900981588496, 0.9985944961894142, 0.9973729422429185, 0.9831028607373031, 0.994387170227028]
[0, 2, 4, 7, 8, 12, 15, 20, 21, 29, 32, 34, 36, 37, 39, 42, 44, 49, 55, 59, 60, 63, 64, 69, 70, 71, 74, 75, 77, 83, 85, 86, 88, 91, 95, 99]
36
36
36


In [20]:
unlabel_2 = unlabel_2.loc[[0, 2, 4, 7, 8, 12, 15, 20, 21, 29, 32, 34, 36, 37, 39, 42, 44, 49, 55, 59, 60, 63, 64, 69, 70, 71, 74, 75, 77, 83, 85, 86, 88, 91, 95, 99],:]
unlabel_2.head()

,Unnamed: 0,Sentence,Target
0,1207,examination occurred night middle may testifie...,Invalid
2,2072,appellant testify offer evidence behalf,Facts
4,906,agreed accompany gave pawn ticket money reclai...,Facts
7,3150,finding door facing street seen man walking pr...,Facts
8,710,appellant indicted robbery indictment alleging...,Analysis


In [21]:
frame_2 = [train4_1,unlabel_2]
train4_2 = pd.concat(frame_2)
len(train4_2)

3551

In [22]:
x_train_2 = tfidf_vect.transform(train4_2['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_2,train4_2['Target'],test_size=0.27,random_state=2)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_2=calibrator.fit(X_train, Y_train)
pred_svm = model_2.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6423357664233577
                  precision    recall  f1-score   support

        Analysis       0.49      0.38      0.43       143
      Conclusion       0.67      0.31      0.42        45
           Facts       0.71      0.90      0.80       485
         Invalid       0.48      0.43      0.46       164
           Issue       0.64      0.33      0.44        63
Rule/Law/Holding       0.56      0.32      0.41        59

        accuracy                           0.64       959
       macro avg       0.59      0.45      0.49       959
    weighted avg       0.62      0.64      0.62       959



In [23]:
x_un3 = tfidf_vect.transform(unlabel_3['Sentence'])
pred_unlabel_3 = model_2.predict_proba(x_un3)

pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_3:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[0.9943821366365208, 0.9926367969737717, 0.9995850477669681, 0.9993186605020636, 0.9994716393707459, 0.9996898563422342, 0.9854428477188736, 0.9950900465240305, 0.9983312588536114, 0.9922341337303849, 0.997489765700114, 0.9989180703978293, 0.9979641711928978, 0.9975304539359945, 0.9943550867283928, 0.999028845258813, 0.9908802527325834, 0.9954513482128862, 0.9969893941939123, 0.9929890676034724, 0.9955277468874464, 0.9967761791792359, 0.9998492924297229, 0.9913438649334296, 0.9997109570436529, 0.9969295666991898, 0.9958393888934858, 0.9994068747240056, 0.9964480624681736, 0.9804614404229594, 0.9994698851327696, 0.9843891993396665, 0.9980552813996774, 0.9934174899666633, 0.9981479507058837, 0.9881206787453028]
[0, 3, 7, 8, 12, 20, 24, 26, 27, 28, 29, 30, 35, 36, 40, 43, 46, 47, 52, 53, 54, 57, 63, 64, 65, 66, 67, 68, 70, 71, 72, 74, 75, 79, 89, 96]
36
36
36


In [24]:
unlabel_3=unlabel_3.loc[[0, 3, 7, 8, 12, 20, 24, 26, 27, 28, 29, 30, 35, 36, 40, 43, 46, 47, 52, 53, 54, 57, 63, 64, 65, 66, 67, 68, 70, 71, 72, 74, 75, 79, 89, 96],:]
frame_3 = [train4_2,unlabel_3]
train4_3 = pd.concat(frame_3)

x_train_3 = tfidf_vect.transform(train4_3['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_3,train4_3['Target'],test_size=0.2,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_3=calibrator.fit(X_train, Y_train)
pred_svm = model_3.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.692200557103064
                  precision    recall  f1-score   support

        Analysis       0.49      0.46      0.47        90
      Conclusion       0.85      0.47      0.61        49
           Facts       0.77      0.90      0.83       382
         Invalid       0.52      0.51      0.51       102
           Issue       0.70      0.42      0.53        45
Rule/Law/Holding       0.51      0.36      0.42        50

        accuracy                           0.69       718
       macro avg       0.64      0.52      0.56       718
    weighted avg       0.68      0.69      0.68       718



In [25]:
x_un4 = tfidf_vect.transform(unlabel_4['Sentence'])
pred_unlabel_4 = model_3.predict_proba(x_un4)


pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_4:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[0, 2, 5, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 5, 2, 2, 2, 2, 2, 2, 2, 2, 5, 2, 2]
[0.9980178494505073, 0.9989771315488409, 0.9871594053462429, 0.9913777737388235, 0.999816350534132, 0.9991454640164746, 0.9835341015740614, 0.9994741264118956, 0.9976430959840119, 0.9985121085969333, 0.9982248466713957, 0.9965857088549521, 0.9995187119029793, 0.9963216666847519, 0.9955231811229978, 0.999049339531099, 0.9992406171363304, 0.9944289623489294, 0.9980576288204611, 0.9993293811706955, 0.991350471330501, 0.9946138831969474, 0.9983785917367338, 0.9936720253157734, 0.9986924503284509, 0.985957719645236, 0.9814432574168686, 0.9972022480355385, 0.9810897999543388, 0.9993587952011326, 0.9984090298435834, 0.999965260184213, 0.9923232277900121, 0.9978062351760515, 0.9941287099734556, 0.981901096129902, 0.9982337772276257, 0.9853500229653618, 0.9880469908175918, 0.9921446968832499]
[2, 4, 6, 7, 10, 15, 16, 18, 21, 22, 27, 28, 29, 31, 32, 34, 37, 38, 39, 40, 46, 49,

In [26]:
unlabel_4=unlabel_4.loc[[2, 4, 6, 7, 10, 15, 16, 18, 21, 22, 27, 28, 29, 31, 32, 34, 37, 38, 39, 40, 46, 49, 50, 55, 56, 60, 64, 67, 68, 69, 70, 75, 80, 85, 87, 89, 93, 94, 97, 99 ],:]

In [27]:
frame_4 = [train4_3,unlabel_4]
train4_4 = pd.concat(frame_4)
len(train4_4)
x_train_4 = tfidf_vect.transform(train4_4['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_4,train4_4['Target'],test_size=0.27,random_state=42)
support = svm.LinearSVC()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_4=calibrator.fit(X_train, Y_train)
pred_svm = model_4.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6704081632653062
                  precision    recall  f1-score   support

        Analysis       0.48      0.41      0.44       146
      Conclusion       0.78      0.54      0.64        52
           Facts       0.78      0.87      0.82       516
         Invalid       0.46      0.49      0.47       136
           Issue       0.57      0.43      0.49        63
Rule/Law/Holding       0.52      0.42      0.46        67

        accuracy                           0.67       980
       macro avg       0.60      0.53      0.55       980
    weighted avg       0.66      0.67      0.66       980



In [28]:
x_un5 = tfidf_vect.transform(unlabel_5['Sentence'])

pred_unlabel_5 = model_4.predict_proba(x_un5)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_5:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 5, 3, 2, 2, 2, 2, 2, 2, 1, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 3, 0]
[0.9989189403942331, 0.9972196787881532, 0.9957306684356463, 0.9941134611279603, 0.9963294010974604, 0.9901576691548221, 0.9987690050741359, 0.9969008275359067, 0.9935207509386235, 0.9914282492120706, 0.9921270860143668, 0.9998370096505548, 0.993344534096129, 0.9946170484649931, 0.9993885982285411, 0.9904020413590066, 0.9976175682882668, 0.9904943693502304, 0.9976422255786858, 0.9867070033751162, 0.9878083702169373, 0.999925537182992, 0.9956281034398644, 0.9941011669119885, 0.9958543657333182, 0.9993147354471515, 0.9988549608947529, 0.9899887570259626, 0.991657585110084, 0.9929132366421051, 0.9853162332378772]
[3, 7, 12, 18, 19, 20, 23, 26, 28, 29, 32, 36, 37, 52, 53, 56, 60, 63, 65, 77, 81, 82, 86, 91, 93, 94, 95, 97, 100, 107, 110]
31
31
31


In [29]:
unlabel_5 = unlabel_5.loc[[3, 7, 12, 18, 19, 20, 23, 26, 28, 29, 32, 36, 37, 52, 53, 56, 60, 63, 65, 77, 81, 82, 86, 91, 93, 94, 95, 97, 100, 107, 110],:]

In [30]:
frame_5 = [train4_4,unlabel_5]
train4_5 = pd.concat(frame_5)
len(train4_5)
x_train_5 = tfidf_vect.transform(train4_5['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_5,train4_5['Target'],test_size=0.2,random_state=42)
support = svm.LinearSVC()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_5=calibrator.fit(X_train, Y_train)
pred_svm = model_5.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6721311475409836
                  precision    recall  f1-score   support

        Analysis       0.42      0.38      0.40        90
      Conclusion       0.66      0.53      0.58        40
           Facts       0.78      0.87      0.82       394
         Invalid       0.47      0.48      0.48       118
           Issue       0.71      0.62      0.67        40
Rule/Law/Holding       0.48      0.28      0.35        50

        accuracy                           0.67       732
       macro avg       0.59      0.53      0.55       732
    weighted avg       0.66      0.67      0.66       732



In [31]:
t_p = tfidf_vect.transform(test['Sentence'])
test_pred = model_5.predict(t_p)
print('Accuracy %s' % accuracy_score(test_pred,test['Target']))

Accuracy 0.6756238003838771


In [32]:
from sklearn.metrics import precision_recall_fscore_support
print(precision_recall_fscore_support(test['Target'],test_pred,average='macro'))

(0.6511776066730416, 0.5832607608422036, 0.6022367446805394, None)
